In [1]:
import pickle

import pandas as pd
import numpy as np

# ignore warnings because they are not relevant
import warnings
warnings.filterwarnings('ignore')

# suppress the scientific notation when printing numpy arrays
np.set_printoptions(suppress=True)

# Loading the data

In [2]:
stock_index = 'SPX'

In [3]:
stocks = pd.read_csv("./data/StockIndices.csv",
                 decimal=',')

# stocks date format: 29/10/2018
stocks['Date'] = pd.to_datetime(stocks['Date'], format='%d/%m/%Y')
stocks.set_index('Date', inplace=True)
stocks.drop(columns='Unnamed: 0', inplace=True)
stocks.head()

# converting prices to floats
stocks['Price Close'] = [float(price) for price in stocks['Price Close']]

# data cleansing
stocks['Index'] = [name.replace(".", "") for name in stocks['Index'].values]

stock_index = stocks[stocks['Index'] == stock_index]
stock_index.head()

,Index,Price Close
Date,,
2008-10-30,SPX,954.09
2008-10-31,SPX,968.75
2008-11-03,SPX,966.30
2008-11-04,SPX,1005.75
2008-11-05,SPX,952.77


In [4]:
weather = pd.read_csv("./data/Weather_ALL.csv",
                 sep=';',
                 decimal=',')

# weather date format: 29/10/2018
weather['Date'] = pd.to_datetime(weather['Date'], format='%d/%m/%Y')
weather.set_index('Date', inplace=True)
weather.head()

# drop NaNs
weather.dropna(inplace=True)

weather.head()

,City,Mean Temperature Actual,Low Temperature Actual,High Temperature Actual,Precipitation Actual,Wind Speed Actual,Relative Humidity Actual
Date,,,,,,,
2018-10-25,Boston,6.64,3.28,10.00,0.10,16.44,69.57
2018-10-24,Boston,7.19,5.00,9.39,0.10,18.07,76.07
2018-10-23,Boston,9.19,5.00,13.30,1.04,13.83,77.78
2018-10-22,Boston,6.65,1.10,12.20,0.00,14.37,49.61
2018-10-21,Boston,7.19,2.80,11.70,0.08,24.00,50.89


# Hypothesis: SPX depends on global weather conditions

In [5]:
# GENERAGE SEASON BINARIES
stock_index['weekday'] = stock_index.index.dayofweek
stock_index['month'] = stock_index.index.month

# Mondays are bad ^^
stock_index['Monday'] = np.where(stock_index['weekday'] == 0, 1, 0)

# Winter is November, December, October
stock_index['Winter'] = np.where((stock_index['month'] == 11) | 
                       (stock_index['month'] == 12) | 
                       (stock_index['month'] == 1), 1, 0)

In [6]:
stock_index.head()

,Index,Price Close,weekday,month,Monday,Winter
Date,,,,,,
2008-10-30,SPX,954.09,3,10,0,0
2008-10-31,SPX,968.75,4,10,0,0
2008-11-03,SPX,966.30,0,11,1,1
2008-11-04,SPX,1005.75,1,11,0,1
2008-11-05,SPX,952.77,2,11,0,1


In [7]:
cities = dict() 
for city in weather['City'].unique():
    
    city_name = city
    
    city = weather[weather['City'] == city].copy()
    
    # GENERATE WEATHER BINARIES
    city['VeryCold'] = np.where(city['Mean Temperature Actual'] <= np.percentile(city['Mean Temperature Actual'], q=25), 1, 0)
    city['HeavyRain'] = np.where(city['Precipitation Actual'] >= np.percentile(city['Precipitation Actual'], q=75), 1, 0)
    city['ColdRain'] = city['HeavyRain']*city['VeryCold']
    


    cities[city_name] = city

In [8]:
cities['London']

,City,Mean Temperature Actual,Low Temperature Actual,High Temperature Actual,Precipitation Actual,Wind Speed Actual,Relative Humidity Actual,VeryCold,HeavyRain,ColdRain
Date,,,,,,,,,,
2018-10-24,London,14.50,11.90,17.11,0.00,13.57,82.90,0,0,0
2018-10-23,London,9.70,4.30,15.11,0.00,13.44,76.33,0,0,0
2018-10-22,London,9.10,5.20,13.00,0.00,13.61,71.08,0,0,0
2018-10-21,London,12.25,5.50,19.00,0.00,6.83,85.63,0,0,0
2018-10-20,London,10.90,3.60,18.22,0.00,4.87,82.87,0,0,0
2018-10-19,London,9.50,2.89,16.00,0.00,4.78,83.42,0,0,0
2018-10-18,London,11.00,6.00,16.00,0.18,12.48,82.65,0,1,0
2018-10-17,London,13.50,11.00,16.00,0.00,5.19,90.47,0,0,0
2018-10-16,London,16.50,13.00,20.00,0.03,11.11,86.26,0,0,0


# Merging prices and weather

In [9]:
data = pd.merge(stock_index, weather, on='Date')